In [90]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import tensorflow as tf 
from tensorflow import keras

In [91]:
df = pd.read_csv("telco_customer.csv")
df.sample(2)

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
4477,6937-GCDGQ,Male,0,Yes,Yes,19,Yes,No,DSL,Yes,...,No,No,No,No,One year,Yes,Bank transfer (automatic),48.95,955.6,No
1447,2951-QOQTK,Male,0,No,Yes,4,Yes,Yes,Fiber optic,No,...,No,Yes,No,No,Month-to-month,No,Credit card (automatic),80.80,332.45,Yes


In [92]:
df.drop('customerID',inplace=True,axis='columns')
df.head(2)

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No


In [93]:
def tableData(df):
    for col in df.columns:
        print(f"Col {col} : {df[col].unique()} , dtype: {df[col].dtype}")

tableData(df)

Col gender : ['Female' 'Male'] , dtype: object
Col SeniorCitizen : [0 1] , dtype: int64
Col Partner : ['Yes' 'No'] , dtype: object
Col Dependents : ['No' 'Yes'] , dtype: object
Col tenure : [ 1 34  2 45  8 22 10 28 62 13 16 58 49 25 69 52 71 21 12 30 47 72 17 27
  5 46 11 70 63 43 15 60 18 66  9  3 31 50 64 56  7 42 35 48 29 65 38 68
 32 55 37 36 41  6  4 33 67 23 57 61 14 20 53 40 59 24 44 19 54 51 26  0
 39] , dtype: int64
Col PhoneService : ['No' 'Yes'] , dtype: object
Col MultipleLines : ['No phone service' 'No' 'Yes'] , dtype: object
Col InternetService : ['DSL' 'Fiber optic' 'No'] , dtype: object
Col OnlineSecurity : ['No' 'Yes' 'No internet service'] , dtype: object
Col OnlineBackup : ['Yes' 'No' 'No internet service'] , dtype: object
Col DeviceProtection : ['No' 'Yes' 'No internet service'] , dtype: object
Col TechSupport : ['No' 'Yes' 'No internet service'] , dtype: object
Col StreamingTV : ['No' 'Yes' 'No internet service'] , dtype: object
Col StreamingMovies : ['No' 'Yes' 'N

#### As we can see Col MultipleLines : ['No phone service' 'No' 'Yes']
We have to morph no phone service to just no
it can be achived using .replace() in panda 

In [94]:
df['MultipleLines'].replace('No phone service','No',inplace=True)
df['OnlineSecurity'].replace('No internet service','No',inplace=True)   #Similarly
df['OnlineBackup'].replace('No internet service','No',inplace=True)   #Similarly
df['DeviceProtection'].replace('No internet service','No',inplace=True)   #Similarly
df['TechSupport'].replace('No internet service','No',inplace=True)   #Similarly
df['StreamingTV'].replace('No internet service','No',inplace=True)   #Similarly
df['StreamingMovies'].replace('No internet service','No',inplace=True)   #Similarly

In [95]:
tableData(df)

Col gender : ['Female' 'Male'] , dtype: object
Col SeniorCitizen : [0 1] , dtype: int64
Col Partner : ['Yes' 'No'] , dtype: object
Col Dependents : ['No' 'Yes'] , dtype: object
Col tenure : [ 1 34  2 45  8 22 10 28 62 13 16 58 49 25 69 52 71 21 12 30 47 72 17 27
  5 46 11 70 63 43 15 60 18 66  9  3 31 50 64 56  7 42 35 48 29 65 38 68
 32 55 37 36 41  6  4 33 67 23 57 61 14 20 53 40 59 24 44 19 54 51 26  0
 39] , dtype: int64
Col PhoneService : ['No' 'Yes'] , dtype: object
Col MultipleLines : ['No' 'Yes'] , dtype: object
Col InternetService : ['DSL' 'Fiber optic' 'No'] , dtype: object
Col OnlineSecurity : ['No' 'Yes'] , dtype: object
Col OnlineBackup : ['Yes' 'No'] , dtype: object
Col DeviceProtection : ['No' 'Yes'] , dtype: object
Col TechSupport : ['No' 'Yes'] , dtype: object
Col StreamingTV : ['No' 'Yes'] , dtype: object
Col StreamingMovies : ['No' 'Yes'] , dtype: object
Col Contract : ['Month-to-month' 'One year' 'Two year'] , dtype: object
Col PaperlessBilling : ['Yes' 'No'] , dtyp

##### Look at TotalCharges column, it is in the form of object, it should be float

In [96]:
df['TotalCharges'].astype('float')

ValueError: could not convert string to float: ' '

As we can see some values are ''

In [97]:
x = pd.to_numeric(df['TotalCharges'], errors='coerce')   # errors coerce converts not changeable to nan
x

0         29.85
1       1889.50
2        108.15
3       1840.75
4        151.65
         ...   
7038    1990.50
7039    7362.90
7040     346.45
7041     306.60
7042    6844.50
Name: TotalCharges, Length: 7043, dtype: float64

In [98]:
df[x.isna()]

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
488,Female,0,Yes,Yes,0,No,No,DSL,Yes,No,Yes,Yes,Yes,No,Two year,Yes,Bank transfer (automatic),52.55,,No
753,Male,0,No,Yes,0,Yes,No,No,No,No,No,No,No,No,Two year,No,Mailed check,20.25,,No
936,Female,0,Yes,Yes,0,Yes,No,DSL,Yes,Yes,Yes,No,Yes,Yes,Two year,No,Mailed check,80.85,,No
1082,Male,0,Yes,Yes,0,Yes,Yes,No,No,No,No,No,No,No,Two year,No,Mailed check,25.75,,No
1340,Female,0,Yes,Yes,0,No,No,DSL,Yes,Yes,Yes,Yes,Yes,No,Two year,No,Credit card (automatic),56.05,,No
3331,Male,0,Yes,Yes,0,Yes,No,No,No,No,No,No,No,No,Two year,No,Mailed check,19.85,,No
3826,Male,0,Yes,Yes,0,Yes,Yes,No,No,No,No,No,No,No,Two year,No,Mailed check,25.35,,No
4380,Female,0,Yes,Yes,0,Yes,No,No,No,No,No,No,No,No,Two year,No,Mailed check,20.00,,No
5218,Male,0,Yes,Yes,0,Yes,No,No,No,No,No,No,No,No,One year,Yes,Mailed check,19.70,,No
6670,Female,0,Yes,Yes,0,Yes,Yes,DSL,No,Yes,Yes,Yes,Yes,No,Two year,No,Mailed check,73.35,,No


#### These are the columns where TotalCharges is ' '

In [99]:
df.iloc[488]['TotalCharges']

' '

#### So we can drop these rows

In [100]:
df = df[df['TotalCharges']!=' ']
df.head(2)

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,Female,0,Yes,No,1,No,No,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No


In [102]:
df['TotalCharges'].isna().unique()

array([False])

In [107]:
df['TotalCharges'] = df['TotalCharges'].astype('float')
tableData(df)

Col gender : ['Female' 'Male'] , dtype: object
Col SeniorCitizen : [0 1] , dtype: int64
Col Partner : ['Yes' 'No'] , dtype: object
Col Dependents : ['No' 'Yes'] , dtype: object
Col tenure : [ 1 34  2 45  8 22 10 28 62 13 16 58 49 25 69 52 71 21 12 30 47 72 17 27
  5 46 11 70 63 43 15 60 18 66  9  3 31 50 64 56  7 42 35 48 29 65 38 68
 32 55 37 36 41  6  4 33 67 23 57 61 14 20 53 40 59 24 44 19 54 51 26 39] , dtype: int64
Col PhoneService : ['No' 'Yes'] , dtype: object
Col MultipleLines : ['No' 'Yes'] , dtype: object
Col InternetService : ['DSL' 'Fiber optic' 'No'] , dtype: object
Col OnlineSecurity : ['No' 'Yes'] , dtype: object
Col OnlineBackup : ['Yes' 'No'] , dtype: object
Col DeviceProtection : ['No' 'Yes'] , dtype: object
Col TechSupport : ['No' 'Yes'] , dtype: object
Col StreamingTV : ['No' 'Yes'] , dtype: object
Col StreamingMovies : ['No' 'Yes'] , dtype: object
Col Contract : ['Month-to-month' 'One year' 'Two year'] , dtype: object
Col PaperlessBilling : ['Yes' 'No'] , dtype: o

But better would be to use pd.to_numeric(df.totalCharges)

#### Lets check if any row has na or null value

In [113]:
for col in df.columns:
    naUnique = df[col].isna().unique()
    nuUnique = df[col].isnull().unique()
    print(f"Col: {col} , na: {naUnique} , null: {nuUnique} ")

Col: gender , na: [False] , null: [False] 
Col: SeniorCitizen , na: [False] , null: [False] 
Col: Partner , na: [False] , null: [False] 
Col: Dependents , na: [False] , null: [False] 
Col: tenure , na: [False] , null: [False] 
Col: PhoneService , na: [False] , null: [False] 
Col: MultipleLines , na: [False] , null: [False] 
Col: InternetService , na: [False] , null: [False] 
Col: OnlineSecurity , na: [False] , null: [False] 
Col: OnlineBackup , na: [False] , null: [False] 
Col: DeviceProtection , na: [False] , null: [False] 
Col: TechSupport , na: [False] , null: [False] 
Col: StreamingTV , na: [False] , null: [False] 
Col: StreamingMovies , na: [False] , null: [False] 
Col: Contract , na: [False] , null: [False] 
Col: PaperlessBilling , na: [False] , null: [False] 
Col: PaymentMethod , na: [False] , null: [False] 
Col: MonthlyCharges , na: [False] , null: [False] 
Col: TotalCharges , na: [False] , null: [False] 
Col: Churn , na: [False] , null: [False] 


### Now data is clean we should scale the data, encode and choose test 

In [115]:
tableData(df)

Col gender : ['Female' 'Male'] , dtype: object
Col SeniorCitizen : [0 1] , dtype: int64
Col Partner : ['Yes' 'No'] , dtype: object
Col Dependents : ['No' 'Yes'] , dtype: object
Col tenure : [ 1 34  2 45  8 22 10 28 62 13 16 58 49 25 69 52 71 21 12 30 47 72 17 27
  5 46 11 70 63 43 15 60 18 66  9  3 31 50 64 56  7 42 35 48 29 65 38 68
 32 55 37 36 41  6  4 33 67 23 57 61 14 20 53 40 59 24 44 19 54 51 26 39] , dtype: int64
Col PhoneService : ['No' 'Yes'] , dtype: object
Col MultipleLines : ['No' 'Yes'] , dtype: object
Col InternetService : ['DSL' 'Fiber optic' 'No'] , dtype: object
Col OnlineSecurity : ['No' 'Yes'] , dtype: object
Col OnlineBackup : ['Yes' 'No'] , dtype: object
Col DeviceProtection : ['No' 'Yes'] , dtype: object
Col TechSupport : ['No' 'Yes'] , dtype: object
Col StreamingTV : ['No' 'Yes'] , dtype: object
Col StreamingMovies : ['No' 'Yes'] , dtype: object
Col Contract : ['Month-to-month' 'One year' 'Two year'] , dtype: object
Col PaperlessBilling : ['Yes' 'No'] , dtype: o

In [117]:
df['gender'].replace({'Female':0, 'Male': 1},inplace=True)
df['Partner'].replace({'Yes':1, 'No':0},inplace=True)
df['Dependents'].replace({'Yes':1, 'No':0},inplace=True)
df['PhoneService'].replace({'Yes':1, 'No':0},inplace=True)
df['MultipleLines'].replace({'Yes':1, 'No':0},inplace=True)
df['OnlineSecurity'].replace({'Yes':1, 'No':0},inplace=True)
df['OnlineBackup'].replace({'Yes':1, 'No':0},inplace=True)
df['DeviceProtection'].replace({'Yes':1, 'No':0},inplace=True)
df['TechSupport'].replace({'Yes':1, 'No':0},inplace=True)
df['StreamingTV'].replace({'Yes':1, 'No':0},inplace=True)
df['StreamingMovies'].replace({'Yes':1, 'No':0},inplace=True)
df['PaperlessBilling'].replace({'Yes':1, 'No':0},inplace=True)
df['Churn'].replace({'Yes':1, 'No':0},inplace=True)

In [118]:
tableData(df)

Col gender : [0 1] , dtype: int64
Col SeniorCitizen : [0 1] , dtype: int64
Col Partner : [1 0] , dtype: int64
Col Dependents : [0 1] , dtype: int64
Col tenure : [ 1 34  2 45  8 22 10 28 62 13 16 58 49 25 69 52 71 21 12 30 47 72 17 27
  5 46 11 70 63 43 15 60 18 66  9  3 31 50 64 56  7 42 35 48 29 65 38 68
 32 55 37 36 41  6  4 33 67 23 57 61 14 20 53 40 59 24 44 19 54 51 26 39] , dtype: int64
Col PhoneService : [0 1] , dtype: int64
Col MultipleLines : [0 1] , dtype: int64
Col InternetService : ['DSL' 'Fiber optic' 'No'] , dtype: object
Col OnlineSecurity : [0 1] , dtype: int64
Col OnlineBackup : [1 0] , dtype: int64
Col DeviceProtection : [0 1] , dtype: int64
Col TechSupport : [0 1] , dtype: int64
Col StreamingTV : [0 1] , dtype: int64
Col StreamingMovies : [0 1] , dtype: int64
Col Contract : ['Month-to-month' 'One year' 'Two year'] , dtype: object
Col PaperlessBilling : [1 0] , dtype: int64
Col PaymentMethod : ['Electronic check' 'Mailed check' 'Bank transfer (automatic)'
 'Credit car

In [119]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [124]:
MmScaler = MinMaxScaler()
df1 = df
df1[['tenure','MonthlyCharges','TotalCharges']] = MmScaler.fit_transform(df[['tenure','MonthlyCharges','TotalCharges']])
tableData(df1)

Col gender : [0 1] , dtype: int64
Col SeniorCitizen : [0 1] , dtype: int64
Col Partner : [1 0] , dtype: int64
Col Dependents : [0 1] , dtype: int64
Col tenure : [0.         0.46478873 0.01408451 0.61971831 0.09859155 0.29577465
 0.12676056 0.38028169 0.85915493 0.16901408 0.21126761 0.8028169
 0.67605634 0.33802817 0.95774648 0.71830986 0.98591549 0.28169014
 0.15492958 0.4084507  0.64788732 1.         0.22535211 0.36619718
 0.05633803 0.63380282 0.14084507 0.97183099 0.87323944 0.5915493
 0.1971831  0.83098592 0.23943662 0.91549296 0.11267606 0.02816901
 0.42253521 0.69014085 0.88732394 0.77464789 0.08450704 0.57746479
 0.47887324 0.66197183 0.3943662  0.90140845 0.52112676 0.94366197
 0.43661972 0.76056338 0.50704225 0.49295775 0.56338028 0.07042254
 0.04225352 0.45070423 0.92957746 0.30985915 0.78873239 0.84507042
 0.18309859 0.26760563 0.73239437 0.54929577 0.81690141 0.32394366
 0.6056338  0.25352113 0.74647887 0.70422535 0.35211268 0.53521127] , dtype: float64
Col PhoneService : 